In [ ]:
import json
import requests
import random
from ipyleaflet import (
    Map,
    LayerGroup,
    SearchControl,
    TileLayer,
    FullScreenControl,
    ScaleControl,
    ZoomControl,
    Marker,
    SearchControl,
    AwesomeIcon,
    WidgetControl,
    GeoJSON,
    Popup,
)
from ipywidgets import IntSlider, ColorPicker, Button, link, HTML, jslink

m = Map(center=(47, 2), zoom=5, zoom_control=False)

# create Zoom_control
m.add(ZoomControl(position="topleft"))
# create Scale_Control
m.add(ScaleControl(position="bottomleft"))
# Create Full_Screen_Control
m.add(FullScreenControl())


zoom_slider = IntSlider(description="Zoom level:", min=0, max=15, value=7)
jslink((zoom_slider, "value"), (m, "zoom"))
widget_control1 = WidgetControl(widget=zoom_slider, position="topright")
m.add(widget_control1)


# Create a local Tile Layer concerning the irsn map
radon = TileLayer(
    url="https://cartoradon.irsn.fr/static/r/{z}/{x}/{y}.png", tms=True, opacity=0.5
)
m.add(radon)


# Create a function get_data which fetch the datas from the irsn application
def get_data(lng, lat):
    r = requests.get(
        "https://cartoradon.irsn.fr/commune.py/communes/lonlat/{};{}".format(lng, lat)
    )
    return r.json()


def generate_popup(coordinates):
    data = get_data(coordinates[1], coordinates[0])
    # Display geojson
    geometry = data["geometry"]
    geojson = GeoJSON(
        data=geometry,
        style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.1, "weight": 1},
        hover_style={"color": "white", "dashArray": "0", "fillOpacity": 0.5},
    )
    m.add(geojson)

    # Create Popup
    message1 = HTML()
    dict_properties = data["properties"]
    get_name = dict_properties.get("name")
    get_class = dict_properties.get("classr")
    message1.value = "Ville: {}, Classe : {}".format(get_name, get_class)
    popup = Popup(
        location=coordinates,
        child=message1,
        close_button=True,
        auto_close=True,
        close_on_escape_key=False,
    )
    m.add(popup)


# take a method on_interation providing Map object to obtain the coordinates
def on_interaction(event, type, coordinates):
    if type == "click":
        generate_popup(coordinates)


m.on_interaction(on_interaction)


def on_found(text, location, **kwargs):
    generate_popup(location)


search = SearchControl(
    position="topleft",
    url="https://cartoradon.irsn.fr/commune.py/communes/search/FR/{s}?",
    zoom=8,
    property_name="display_name",
)
search.on_location_found(on_found)

m.add(search)

In [ ]:
from ipyflex import FlexLayout

w = {"dashboard": m}

In [ ]:
a = FlexLayout(w, template="kk.json", editable=False)

In [ ]:
a

In [ ]:
slider = IntSlider()

In [ ]:
a.add("hh", slider)